# Generating dispense list for external barcodes for Echo liquid handler
The script takes generated barcodes list per file (`barcodes_by_file.csv`), reads it against plate specifications of an IDT plate containing the external barcodes (`Plate Specs.xlsx`), and generate two output files: `extBarcode_dispense_list.csv` and `extBarcode_dispense_list_check.csv`.

`extBarcode_dispense_list.csv` provides a csv file that can be uploaded directly on Echo liquid handler. `extBarcode_dispense_list_check.csv` contains a list of barcodes that goes for each well for cross-validation.

In [1]:
import pandas as pd
import csv
import string

# Pad csv file because column elements are not the same for all rows
x = []
with open(r'barcodes_by_file.csv') as csvfile:
    reader = csv.reader(csvfile)
    max_elem = 0
    for row in reader:
        if max_elem < len(row):
            max_elem = len(row)
        csvfile.seek(0)
        for i,row in enumerate(reader):
            x.append(row + ['' for x in range(max_elem-len(row))])

barcodes_list = pd.DataFrame(x)
                   
plate = pd.read_excel(r'Plate Specs.xlsx')
barcodes = plate.loc[plate['Plate Name']=='Ext_Barcodes_v2']

w = list(string.ascii_uppercase)

dispense_list = []
dispense_list_check = []
v_dispense = 50
l = 0
for i in range(barcodes_list.shape[0]):
    if i == 0:
        c = 1
        l = 0
    elif i % 12 == 0:
        c = 1
        l += 1
    else: 
        c += 1
    for j in range(1,barcodes_list.shape[1]):
        if barcodes_list.iloc[i,j] == None:
            continue
        else:
            barcode_plate_address = plate.loc[plate['Sequence Name'] == barcodes_list.iloc[i,j]]['Well Position'].iloc[0]
            if barcode_plate_address[1] == '0':
                barcode_plate_address = barcode_plate_address.replace('0','')
            dispense_list.append(['Ext_Barcodes_v2',barcode_plate_address,'Destination',w[l] + str(c),str(v_dispense)])
            dispense_list_check.append(['Ext_Barcodes_v2',barcode_plate_address,barcodes_list.iloc[i,j]])
        
dispense_csv = 'extBarcode_dispense_list.csv'
dispense_check_csv = 'extBarcode_dispense_list_check.csv'
headers = ['Source Plate Barcode','Source Well','Destination Plate Barcode',\
            'Destination Well','Transfer Volume']   

with open(dispense_csv,'w',newline='') as csvfile:
    csvwriter = csv.writer(csvfile) 
    csvwriter.writerow(headers)
    csvwriter.writerows(dispense_list)
    
with open(dispense_check_csv,'w',newline='') as csvfile:
    csvwriter = csv.writer(csvfile) 
    csvwriter.writerows(dispense_list_check)